# M3 Monthly Data: Load + Initial Cleaning

This notebook loads M3 monthly actual values (`TSTS`) and forecaster values (`FTS`) using the project loading module, then performs a minimal first-pass cleaning scaffold.

In [22]:
import sys
from pathlib import Path

import pandas as pd

PROJECT_ROOT = Path.cwd().resolve().parent if Path.cwd().name == 'analyses' else Path.cwd().resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.data.loading import (
    DEFAULT_M3_QUARTERLY_ACTUALS_CSV_PATH,
    DEFAULT_M3_QUARTERLY_FORECASTS_CSV_PATH,
    load_or_download_m3_quarterly_actuals,
    load_or_download_m3_quarterly_forecasts,
)
from src.data.cleaning import (
    align_m3_monthly_actuals_and_forecasts,
    build_m3_series_horizon_matrix,
    prepare_m3_monthly_data,
)

# Monthly aliases for readability (loader names remain backward-compatible).
M3_MONTHLY_ACTUALS_CSV_PATH = DEFAULT_M3_QUARTERLY_ACTUALS_CSV_PATH
M3_MONTHLY_FORECASTS_CSV_PATH = DEFAULT_M3_QUARTERLY_FORECASTS_CSV_PATH


In [23]:
actuals_raw = load_or_download_m3_quarterly_actuals()
forecasts_raw = load_or_download_m3_quarterly_forecasts()

print('Cached actuals path:', M3_MONTHLY_ACTUALS_CSV_PATH)
print('Cached forecasts path:', M3_MONTHLY_FORECASTS_CSV_PATH)
print('actuals_raw shape:', actuals_raw.shape)
print('forecasts_raw shape:', forecasts_raw.shape)

display(actuals_raw.head())
display(forecasts_raw.head())


Cached actuals path: /home/clayt/Ensemble-Forecasting/data/M3_monthly_TSTS.csv
Cached forecasts path: /home/clayt/Ensemble-Forecasting/data/M3_monthly_FTS.csv
actuals_raw shape: (167562, 4)
forecasts_raw shape: (616896, 6)


,series_id,category,value,timestamp
0,M1,MICRO,2640.0,1990-01
1,M1,MICRO,2640.0,1990-02
2,M1,MICRO,2160.0,1990-03
3,M1,MICRO,4200.0,1990-04
4,M1,MICRO,3360.0,1990-05


,series_id,method_id,forecast,horizon,timestamp,origin_timestamp
0,M1,NAIVE2,2400.0,1,1994-03,1994-02
1,M1,NAIVE2,2400.0,2,1994-04,1994-02
2,M1,NAIVE2,2400.0,3,1994-05,1994-02
3,M1,NAIVE2,2400.0,4,1994-06,1994-02
4,M1,NAIVE2,2400.0,5,1994-07,1994-02


In [24]:
# Keep only macro series and align actuals with forecasts by horizon
actuals_macro, forecasts_macro, macro_series_ids = prepare_m3_monthly_data(
    actuals_raw, forecasts_raw, category='MACRO'
)

# Enforce expected macro block and natural numeric ordering (M809..M1120).
macro_series_ids = pd.Index(
    sorted(
        [
            sid for sid in map(str, macro_series_ids.tolist())
            if sid.startswith('M') and sid[1:].isdigit() and 809 <= int(sid[1:]) <= 1120
        ],
        key=lambda s: int(s[1:]),
    ),
    name='series_id',
)
actuals_macro = actuals_macro.loc[actuals_macro['series_id'].isin(macro_series_ids)].copy()
forecasts_macro = forecasts_macro.loc[forecasts_macro['series_id'].isin(macro_series_ids)].copy()

aligned_long = align_m3_monthly_actuals_and_forecasts(actuals_macro, forecasts_macro)
aligned_long = aligned_long.loc[aligned_long['series_id'].isin(macro_series_ids)].copy()

print('Macro series count:', len(macro_series_ids))
print('First 5 macro IDs:', macro_series_ids[:5].tolist())
print('Last 5 macro IDs:', macro_series_ids[-5:].tolist())
print('actuals_macro shape:', actuals_macro.shape)
print('forecasts_macro shape:', forecasts_macro.shape)
print('aligned_long shape:', aligned_long.shape)
print('Horizon consistency share:', aligned_long['horizon_consistent'].mean())

display(actuals_macro.head())
display(forecasts_macro.head())
display(aligned_long.head())



Macro series count: 312
First 5 macro IDs: ['M809', 'M810', 'M811', 'M812', 'M813']
Last 5 macro IDs: ['M1116', 'M1117', 'M1118', 'M1119', 'M1120']
actuals_macro shape: (40835, 6)
forecasts_macro shape: (134784, 8)
aligned_long shape: (134784, 9)
Horizon consistency share: 1.0


,series_id,category,value,timestamp,actual,period
115764,M1000,MACRO,3705.4,1983-01,3705.4,1983-01
115765,M1000,MACRO,3726.0,1983-02,3726.0,1983-02
115766,M1000,MACRO,3692.0,1983-03,3692.0,1983-03
115767,M1000,MACRO,3721.6,1983-04,3721.6,1983-04
115768,M1000,MACRO,3681.0,1983-05,3681.0,1983-05


,series_id,method_id,forecast,horizon,timestamp,origin_timestamp,target_period,origin_period
431964,M1000,AAM1,4556.58,1,1992-09,1992-08,1992-09,1992-08
431965,M1000,AAM1,4571.48,2,1992-10,1992-08,1992-10,1992-08
431966,M1000,AAM1,4577.80,3,1992-11,1992-08,1992-11,1992-08
431967,M1000,AAM1,4586.75,4,1992-12,1992-08,1992-12,1992-08
431968,M1000,AAM1,4594.23,5,1993-01,1992-08,1993-01,1992-08


,series_id,method_id,horizon,origin_period,target_period,expected_target_period,horizon_consistent,forecast,actual
0,M1000,AAM1,1,1992-08,1992-09,1992-09,True,4556.58,4580.6
1,M1000,AAM1,2,1992-08,1992-10,1992-10,True,4571.48,4563.4
2,M1000,AAM1,3,1992-08,1992-11,1992-11,True,4577.80,4551.8
3,M1000,AAM1,4,1992-08,1992-12,1992-12,True,4586.75,4577.4
4,M1000,AAM1,5,1992-08,1993-01,1993-01,True,4594.23,4592.4


In [25]:
# Build an ensemble-ready matrix for one macro series and one horizon
sample_series_id = 'M809' if 'M809' in set(map(str, macro_series_ids.tolist())) else str(macro_series_ids[0])
sample_horizon = int(aligned_long['horizon'].dropna().astype(int).min())

series_h_matrix = build_m3_series_horizon_matrix(
    aligned_df=aligned_long,
    series_id=sample_series_id,
    horizon=sample_horizon,
    require_actual=True,
)

print('sample_series_id:', sample_series_id)
print('sample_horizon:', sample_horizon)
print('series_h_matrix shape:', series_h_matrix.shape)
display(series_h_matrix.head())

# This matrix is now ready for ensemble experiments:
# - target: series_h_matrix['actual']
# - experts: method columns (all columns except origin_period, target_period, actual)



sample_series_id: M809
sample_horizon: 1
series_h_matrix shape: (1, 27)


,origin_period,actual,target_period,AAM1,AAM2,ARARMA,Auto-ANN,AutoBox1,AutoBox2,AutoBox3,...,HOLT,NAIVE2,PP-Autocast,RBF,ROBUST-Trend,SINGLE,SMARTFCS,THETA,THETAsm,WINTER
0,1992-08,5172.4,1992-09,5145.84,5145.3,5144.66,5145.55,5168.47,5138.22,5169.61,...,5144.73,5118.4,5150.17,5175.97,5138.57,5118.4,5157.14,5138.96,5133.24,5144.73


In [26]:
# M809 first: run sequential ensembling across horizons (competition layout)
import importlib
import pandas as pd
import numpy as np
import src.evaluation.m3_macro_experiment as m3exp
m3exp = importlib.reload(m3exp)
from src.evaluation import optuna_tuning as ot

series_online_data_fixed_methods = m3exp.series_online_data_fixed_methods
aggregate = m3exp.aggregate
evaluate_series_horizon = m3exp.evaluate_series_horizon

SERIES_ID = 'M809'

actuals_m809 = actuals_macro.loc[actuals_macro['series_id'] == SERIES_ID].copy()
fc_m809 = forecasts_macro.loc[forecasts_macro['series_id'] == SERIES_ID].copy()
aligned_m809 = aligned_long.loc[aligned_long['series_id'] == SERIES_ID].copy()

print('M809 actual period range:', actuals_m809['period'].min(), 'to', actuals_m809['period'].max())
print('M809 forecast origin range:', fc_m809['origin_period'].min(), 'to', fc_m809['origin_period'].max())
print('M809 target range:', fc_m809['target_period'].min(), 'to', fc_m809['target_period'].max())

methods_m809 = sorted(fc_m809['method_id'].dropna().astype(str).unique().tolist())
print('M809 method count:', len(methods_m809))
print('M809 methods:', methods_m809)

LOSS_SECTIONS = ['mse', 'linex']
LINEX_A = 1.0
INCLUDE_RL = True
MIN_OBS = 6
MIN_METHODS = 3

kappa_grid = np.array([0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 8.0], dtype=float)
params_map = {k: dict(v) for k, v in ot.DEFAULT_METHOD_PARAMS.items()}

d = series_online_data_fixed_methods(
    aligned_long=aligned_m809,
    series_id=SERIES_ID,
    required_methods=methods_m809,
    min_obs=int(MIN_OBS),
    min_methods=int(MIN_METHODS),
)

if d is None:
    raise RuntimeError('Could not build M809 online panel. Check alignment/method coverage.')

print('M809 selected origin:', d['origin_period'])
print('M809 panel shape:', d['mat'].shape)
print('M809 horizons in panel:', d['mat']['horizon'].tolist())

detailed_rows_m809 = []
diag_rows_m809 = []
for loss_section in LOSS_SECTIONS:
    rows_h, diag_h = evaluate_series_horizon(
        series_id=SERIES_ID,
        horizon=0,
        y=d['y'],
        F=d['F'],
        s=d['s'],
        loss_section=loss_section,
        linex_a=float(LINEX_A),
        include_rl=bool(INCLUDE_RL),
        params_map=params_map,
        kappa_grid=kappa_grid,
    )
    detailed_rows_m809.extend(rows_h)
    diag_rows_m809.extend(diag_h)

summary_rows_m809 = aggregate(detailed_rows_m809)
m809_summary_df = pd.DataFrame(summary_rows_m809)
if not m809_summary_df.empty:
    m809_summary_df = m809_summary_df.sort_values(['loss_section', 'objective_mean']).reset_index(drop=True)

print('M809 detailed rows:', len(detailed_rows_m809))
print('M809 diagnostic rows:', len(diag_rows_m809))
display(m809_summary_df)



M809 actual period range: 1983-01 to 1994-02
M809 forecast origin range: 1992-08 to 1992-08
M809 target range: 1992-09 to 1994-02
M809 method count: 24
M809 methods: ['AAM1', 'AAM2', 'ARARMA', 'Auto-ANN', 'AutoBox1', 'AutoBox2', 'AutoBox3', 'B-J auto', 'COMB S-H-D', 'DAMPEN', 'Flors-Pearc1', 'Flors-Pearc2', 'ForcX', 'ForecastPro', 'HOLT', 'NAIVE2', 'PP-Autocast', 'RBF', 'ROBUST-Trend', 'SINGLE', 'SMARTFCS', 'THETA', 'THETAsm', 'WINTER']
M809 selected origin: 1992-08
M809 panel shape: (18, 27)
M809 horizons in panel: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
M809 detailed rows: 20
M809 diagnostic rows: 72


,loss_section,horizon,method,n_series,objective_mean,objective_std,mse_mean,mae_mean,linex_mean,avg_hhi_mean,avg_lambda_mean,avg_excess_objective_vs_best_individual,avg_improvement_pct_vs_best_individual
0,linex,0.0,MWUMBothKL,1.0,6.676099e+101,0.0,4341.290211,42.127932,6.676099e+101,0.941088,0.323685,0.000000e+00,0.000000e+00
1,linex,0.0,MWUMConcOnlyKL,1.0,6.676099e+101,0.0,4313.892776,42.367023,6.676099e+101,0.891043,0.323685,0.000000e+00,0.000000e+00
2,linex,0.0,MWUMVanilla,1.0,6.676099e+101,0.0,4341.033238,42.124492,6.676099e+101,0.941591,NaN,0.000000e+00,0.000000e+00
3,linex,0.0,RLKappaBandit,1.0,6.676099e+101,0.0,4335.424888,42.856872,6.676099e+101,0.772501,2.034712,0.000000e+00,0.000000e+00
4,linex,0.0,Median,1.0,7.836061e+128,0.0,9157.638253,71.608333,7.836061e+128,NaN,NaN,7.836061e+128,-1.173749e+29
5,linex,0.0,Mean,1.0,1.401063e+131,0.0,16033.455913,106.356181,1.401063e+131,0.041667,NaN,1.401063e+131,-2.098625e+31
6,linex,0.0,OGDBoth,1.0,1.401063e+131,0.0,17522.657558,113.966991,1.401063e+131,0.173611,0.323685,1.401063e+131,-2.098625e+31
7,linex,0.0,OGDConcOnly,1.0,1.401063e+131,0.0,17781.224831,116.231227,1.401063e+131,0.120370,0.323685,1.401063e+131,-2.098625e+31
8,linex,0.0,OGDVanilla,1.0,1.401063e+131,0.0,17522.657558,113.966991,1.401063e+131,0.173611,NaN,1.401063e+131,-2.098625e+31
9,linex,0.0,RLRuleBandit,1.0,1.401063e+131,0.0,10635.139066,65.591547,1.401063e+131,0.458302,NaN,1.401063e+131,-2.098625e+31


In [27]:
# Scale across ALL macro series: sequential ensembling across horizons per series
import importlib
import numpy as np
import pandas as pd
import src.evaluation.m3_macro_experiment as m3exp
m3exp = importlib.reload(m3exp)
from src.evaluation import optuna_tuning as ot

series_online_data_fixed_methods = m3exp.series_online_data_fixed_methods
aggregate = m3exp.aggregate
evaluate_series_horizon = m3exp.evaluate_series_horizon
write_csv = m3exp.write_csv
write_report = m3exp.write_report

LOSS_SECTIONS_ALL = ['mse', 'linex']
LINEX_A_ALL = 1.0
INCLUDE_RL_ALL = True
MIN_OBS_ALL = 6
MIN_METHODS_ALL = 3

series_ids_all = sorted(list(map(str, macro_series_ids.tolist())), key=lambda s: int(s[1:]) if (isinstance(s, str) and s.startswith('M') and s[1:].isdigit()) else 10**9)

kappa_grid = np.array([0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 8.0], dtype=float)
params_map = {k: dict(v) for k, v in ot.DEFAULT_METHOD_PARAMS.items()}

detailed_rows_all = []
diag_rows_all = []
skip_counts_all = {'no_panel': 0}

for loss_section in LOSS_SECTIONS_ALL:
    for sid in series_ids_all:
        sid_fc = forecasts_macro.loc[forecasts_macro['series_id'] == sid]
        methods_sid = sorted(sid_fc['method_id'].dropna().astype(str).unique().tolist())
        d = series_online_data_fixed_methods(
            aligned_long=aligned_long,
            series_id=sid,
            required_methods=methods_sid,
            min_obs=int(MIN_OBS_ALL),
            min_methods=int(MIN_METHODS_ALL),
        )
        if d is None:
            skip_counts_all['no_panel'] += 1
            continue
        rows_h, diag_h = evaluate_series_horizon(
            series_id=sid,
            horizon=0,
            y=d['y'],
            F=d['F'],
            s=d['s'],
            loss_section=loss_section,
            linex_a=float(LINEX_A_ALL),
            include_rl=bool(INCLUDE_RL_ALL),
            params_map=params_map,
            kappa_grid=kappa_grid,
        )
        detailed_rows_all.extend(rows_h)
        diag_rows_all.extend(diag_h)

summary_rows_all = aggregate(detailed_rows_all)
summary_df_all = pd.DataFrame(summary_rows_all)
if not summary_df_all.empty:
    summary_df_all = summary_df_all.sort_values(['loss_section', 'objective_mean']).reset_index(drop=True)

print('All-series detailed rows:', len(detailed_rows_all))
print('All-series diagnostics rows:', len(diag_rows_all))
print('Macro series evaluated:', len(set(r['series_id'] for r in detailed_rows_all)) if detailed_rows_all else 0)
print('All-series skip counts:', skip_counts_all)
display(summary_df_all)

OUT_DIR = PROJECT_ROOT / 'analyses' / 'results' / 'm3_macro'
OUT_STEM = 'm3_macro_full'
OUT_DIR.mkdir(parents=True, exist_ok=True)

detailed_csv = OUT_DIR / f'{OUT_STEM}_detailed.csv'
summary_csv = OUT_DIR / f'{OUT_STEM}_summary.csv'
diag_csv = OUT_DIR / f'{OUT_STEM}_policy_diagnostics.csv'
report_md = OUT_DIR / f'{OUT_STEM}_report.md'

write_csv(detailed_csv, detailed_rows_all)
write_csv(summary_csv, summary_rows_all)
write_csv(diag_csv, diag_rows_all)
write_report(report_md, summary_rows_all, detailed_rows_all, linex_a=float(LINEX_A_ALL))

print('Wrote:', detailed_csv)
print('Wrote:', summary_csv)
print('Wrote:', diag_csv)
print('Wrote:', report_md)




/home/clayt/Ensemble-Forecasting/src/evaluation/evaluation_helpers.py:42: RuntimeWarning: overflow encountered in exp
  L = np.exp(a * e) - a * e - 1.0
/home/clayt/Ensemble-Forecasting/src/evaluation/evaluation_helpers.py:42: RuntimeWarning: overflow encountered in exp
  L = np.exp(a * e) - a * e - 1.0
/home/clayt/Ensemble-Forecasting/src/evaluation/evaluation_helpers.py:42: RuntimeWarning: overflow encountered in exp
  L = np.exp(a * e) - a * e - 1.0
/home/clayt/Ensemble-Forecasting/src/evaluation/evaluation_helpers.py:42: RuntimeWarning: overflow encountered in exp
  L = np.exp(a * e) - a * e - 1.0
/home/clayt/Ensemble-Forecasting/src/evaluation/evaluation_helpers.py:42: RuntimeWarning: overflow encountered in exp
  L = np.exp(a * e) - a * e - 1.0
/home/clayt/Ensemble-Forecasting/src/evaluation/evaluation_helpers.py:42: RuntimeWarning: overflow encountered in exp
  L = np.exp(a * e) - a * e - 1.0
/home/clayt/Ensemble-Forecasting/src/evaluation/evaluation_helpers.py:42: RuntimeWarning

All-series detailed rows: 6020
All-series diagnostics rows: 21372
Macro series evaluated: 301
All-series skip counts: {'no_panel': 22}


/home/clayt/Ensemble-Forecasting/src/ensemblers/ensemblers.py:531: RuntimeWarning: invalid value encountered in matmul
  avg_loss[t] = float(w @ ell)
/home/clayt/Ensemble-Forecasting/src/ensemblers/ensemblers.py:543: RuntimeWarning: invalid value encountered in subtract
  logw_new -= np.max(logw_new)
/home/clayt/Ensemble-Forecasting/src/ensemblers/ensemblers.py:601: RuntimeWarning: invalid value encountered in matmul
  avg_loss[t] = float(w @ ell)
/home/clayt/Ensemble-Forecasting/src/ensemblers/ensemblers.py:614: RuntimeWarning: invalid value encountered in subtract
  logw -= np.max(logw)
/home/clayt/Ensemble-Forecasting/src/ensemblers/rl.py:128: RuntimeWarning: overflow encountered in exp
  return float(self.linex_a * (np.exp(self.linex_a * e) - 1.0))
/home/clayt/Ensemble-Forecasting/src/ensemblers/rl.py:14: RuntimeWarning: overflow encountered in exp
  return np.exp(a * e) - a * e - 1.0
/home/clayt/Ensemble-Forecasting/src/ensemblers/rl.py:38: RuntimeWarning: overflow encountered in 

,loss_section,horizon,method,n_series,objective_mean,objective_std,mse_mean,mae_mean,linex_mean,avg_hhi_mean,avg_lambda_mean,avg_excess_objective_vs_best_individual,avg_improvement_pct_vs_best_individual
0,linex,0.0,MWUMBothKL,301.0,inf,NaN,4.672742e+05,301.524162,inf,0.855202,0.411272,NaN,-1.355848e+277
1,linex,0.0,MWUMConcOnlyKL,301.0,inf,NaN,4.445380e+05,264.204486,inf,0.879880,0.411272,NaN,-1.378992e+254
2,linex,0.0,MWUMVanilla,301.0,inf,NaN,4.688806e+05,306.559698,inf,0.861963,NaN,NaN,-1.355848e+277
3,linex,0.0,Mean,301.0,inf,NaN,6.966272e+05,421.936898,inf,0.041667,NaN,NaN,-1.302741e+294
4,linex,0.0,Median,301.0,inf,NaN,6.832493e+05,428.205353,inf,NaN,NaN,NaN,-1.136535e+279
5,linex,0.0,OGDBoth,301.0,inf,NaN,7.958330e+05,412.209736,inf,0.296403,0.411272,NaN,-4.638147e+283
6,linex,0.0,OGDConcOnly,301.0,inf,NaN,7.951320e+05,411.267257,inf,0.326535,0.411272,NaN,-4.638147e+283
7,linex,0.0,OGDVanilla,301.0,inf,NaN,7.957804e+05,412.219840,inf,0.297000,NaN,NaN,-4.638147e+283
8,linex,0.0,RLKappaBandit,301.0,inf,NaN,4.445700e+05,264.201065,inf,0.876221,0.882456,NaN,-1.989358e+252
9,linex,0.0,RLRuleBandit,301.0,inf,NaN,6.329310e+05,365.489417,inf,0.367873,NaN,NaN,-2.637016e+287


Wrote: /home/clayt/Ensemble-Forecasting/analyses/results/m3_macro/m3_macro_full_detailed.csv
Wrote: /home/clayt/Ensemble-Forecasting/analyses/results/m3_macro/m3_macro_full_summary.csv
Wrote: /home/clayt/Ensemble-Forecasting/analyses/results/m3_macro/m3_macro_full_policy_diagnostics.csv
Wrote: /home/clayt/Ensemble-Forecasting/analyses/results/m3_macro/m3_macro_full_report.md
